In [1]:
import pandas as pd

In [2]:
green_prefix = 'https://d37ci6vzurychx.cloudfront.net/trip-data' 
green_url = f'{green_prefix}/green_tripdata_2025-11.parquet'
green_url

'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet'

In [3]:
zones_prefix = 'https://github.com/DataTalksClub/nyc-tlc-data'
zones_url = f'{zones_prefix}/releases/download/misc/taxi_zone_lookup.csv'
zones_url

'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv'

In [4]:
green_dtypes = {
    "VendorID":"Int64",
    "store_and_fwd_flag":
    

In [13]:
print(df_green.dtypes)

VendorID                          int32
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag                  str
RatecodeID                      float64
PULocationID                      int32
DOLocationID                      int32
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                       float64
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
cbd_congestion_fee              float64
dtype: object


In [9]:
print(df_green.head(5))

   VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
0         2  2025-11-01 00:34:48   2025-11-01 00:41:39                  N   
1         2  2025-11-01 00:18:52   2025-11-01 00:24:27                  N   
2         2  2025-11-01 01:03:14   2025-11-01 01:15:24                  N   
3         2  2025-11-01 00:10:57   2025-11-01 00:24:53                  N   
4         1  2025-11-01 00:03:48   2025-11-01 00:19:38                  N   

   RatecodeID  PULocationID  DOLocationID  passenger_count  trip_distance  \
0         1.0            74            42              1.0           0.74   
1         1.0            74            42              2.0           0.95   
2         1.0            83           160              1.0           2.19   
3         1.0           166           127              1.0           5.44   
4         1.0           166           262              1.0           3.20   

   fare_amount  ...  mta_tax  tip_amount  tolls_amount  ehail_fee  \
0    